In [ ]:
%sql
-- Executive summary: Risk Queue by Building
CREATE OR REPLACE VIEW fmucd_capstone.gold.v_risk_by_building AS
SELECT
  building_id,
  building_name,
  state_province,
  COUNT(*) AS work_orders,
  AVG(risk_score) AS avg_risk_score,
  SUM(total_cost) AS total_cost
FROM fmucd_capstone.gold.high_duration_risk_queue_ranked
WHERE risk_bucket_ranked IN ('HIGH','MEDIUM')
GROUP BY 1,2,3
ORDER BY avg_risk_score DESC, total_cost DESC;

In [ ]:
%sql
-- Risk by system
CREATE OR REPLACE VIEW fmucd_capstone.gold.v_risk_by_system AS
SELECT
  COALESCE(system_code,'UNKNOWN') AS system_code,
  COALESCE(system_description,'Unclassified') AS system_description,
  COUNT(*) AS work_orders,
  AVG(risk_score) AS avg_risk_score,
  SUM(total_cost) AS total_cost
FROM fmucd_capstone.gold.high_duration_risk_queue_ranked
WHERE risk_bucket_ranked IN ('HIGH','MEDIUM')
GROUP BY 1,2
ORDER BY avg_risk_score DESC, total_cost DESC;

In [ ]:
%sql
-- Ops queue: top 500 HIGH by score
CREATE OR REPLACE VIEW fmucd_capstone.gold.v_ops_queue_top500 AS
SELECT
  wo_id,
  wo_priority,
  wo_start_date,
  wo_end_date,
  wo_duration_days,
  building_id,
  building_name,
  COALESCE(system_code,'UNKNOWN') AS system_code,
  COALESCE(system_description,'Unclassified') AS system_description,
  total_cost,
  labor_hours,
  risk_score,
  risk_bucket_ranked
FROM fmucd_capstone.gold.high_duration_risk_queue_ranked
WHERE risk_bucket_ranked = 'HIGH'
ORDER BY risk_score DESC
LIMIT 500;

In [ ]:
%sql
SELECT risk_bucket_ranked, COUNT(*) 
FROM fmucd_capstone.gold.high_duration_risk_queue_ranked
GROUP BY risk_bucket_ranked
ORDER BY 2 DESC;